###Imports

In [1]:
import keras
from keras.layers import Conv2D, BatchNormalization, MaxPool2D, GlobalMaxPool2D , Flatten
from keras.layers import TimeDistributed, LSTM, Dense, Dropout
import numpy as np
import datetime
import tensorflow as tf
import pandas as pd
import h5py
from keras import backend as k
from keras.callbacks import ModelCheckpoint

#tf.debugging.set_log_device_placement(True)

The convNet:

In [2]:
def video_generator(path , batch_size = 1 , train_data_size = 180 , test_data_size = 16 , mode = "train"):

    count = 1
    #infinite loop
    if(mode == "train"):
        while True:
            #check if it is the end of train data size
            #if its the end, set the counter to 1 for re-iteration
            if(count > train_data_size):
                count = 1
                
            #read hdf5
            file = h5py.File(path + "video (" + str(count) + ").hdf5" , "r+")
            video = np.array(file["/video"]).astype("uint8").reshape((1 , 300 , 360 , 640 , 3))
            label = np.array(file["/label"]).astype("uint8").reshape((1 , 1))

            #label = np.log(label)
            
            #print("Now training on example " + str(count))
            count = count + 1
            
            #yield the input
            yield (video , label)

    elif(mode == "eval"):
        while True:
            #check if it is the end of train data size
            #if its the end, set the counter to 1 for re-iteration
            if(count > test_data_size):
                count = 1
                
            #read hdf5
            file = h5py.File(path + "video (" + str(count + train_data_size) + ").hdf5" , "r+")
            video = np.array(file["/video"]).astype("uint8").reshape((1 , 300 , 360 , 640 , 3))
            label = np.array(file["/label"]).astype("uint8").reshape((1 , 1))

            #label = np.log(label)

            print("Now testing on example " + str(count))

            count = count + 1
            
            #yield the input
            yield (video , label)

In [3]:
def single_convnet(shape):
    momentum = .9
    model = keras.Sequential()
    #model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(32, (3,3), strides=(2 , 2) , input_shape=shape, padding='valid', activation='relu' , name = "conv_1"))
    model.add(Dropout(0.2))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    #model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    #model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    #model.add(BatchNormalization(momentum=momentum))
    
    #model.add(MaxPool2D())
    
    #model.add(Conv2D(256, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(32, (3,3), strides=(2 , 2) , padding='valid', activation='relu' , name = "conv_2"))
    model.add(Dropout(0.2))
    model.add(BatchNormalization(momentum=momentum))
    
    model.add(MaxPool2D())
    
    #model.add(Conv2D(1024, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(32, (1,1) , strides=(2 , 2) , padding='valid', activation='relu' , name = "conv_3"))
    model.add(Dropout(0.2))
    model.add(BatchNormalization(momentum=momentum))
    model.add(MaxPool2D())
    
    # flatten...
    #model.add(GlobalMaxPool2D())
    model.add(Flatten())
    model.add(Dense((256) , activation="relu"))
    model.summary()
    return model

In [4]:
def Model(shape):
    # Create our convnet with (256 , 144 , 3) input shape
    convnet = single_convnet(shape[1:])
    
    # then create our final model
    model = keras.Sequential()
    # add the convnet with (200 , 256 , 144 , 3) shape
    model.add(TimeDistributed(convnet, input_shape=shape))
    # here, you can also use GRU or LSTM
    model.add(LSTM(32))
    # and finally, we make a decision network
    #model.add(Dense(1024, activation='relu'))
    #model.add(Dropout(.5))
    #model.add(Dense(512, activation='relu'))
    #model.add(Dropout(.5))
    #model.add(Dense(128, activation='relu'))
    #model.add(Dropout(.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(32 , activation='relu'))
    model.add(Dense(1))
    return model

In [5]:
#load the already existing model along with weights etc.
#model = keras.models.load_model("model")

#Else compile the model
model = Model((300 , 360 , 640 , 3))
model.summary()
#load weights if there are saved weights, loads best weights
#model.load_weights("model_1/variables/variables")
#load weights if there are saved weights, loads last weights
model.load_weights("model_1")
model.compile(loss=keras.losses.MeanSquaredError(), optimizer='adam', metrics=['mse'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 179, 319, 32)      896       
_________________________________________________________________
dropout (Dropout)            (None, 179, 319, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 179, 319, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 89, 159, 32)       0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 44, 79, 32)        9248      
_________________________________________________________________
dropout_1 (Dropout)          (None, 44, 79, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 44, 79, 32)        1

In [6]:
#this cell is for testing the model with mock data only
#do not use in the actual implementation
x = np.ones((1 , 300 , 360 , 640 , 3))
y = np.array([.25])
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)



with tf.device("/CPU:0"):
    model.fit(x , y , epochs = 50 , callbacks=[tensorboard_callback])

Epoch 1/50
1/1 [==============================] - 0s 454ms/step - loss: 0.0081 - mse: 0.0081
Epoch 2/50


In [13]:
#This cell is for training
#setting up tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#setting up checkpoint
checkpoint = ModelCheckpoint("model_1", monitor='loss', verbose=1, save_best_only=True, mode='auto', save_freq=20)

#setting the path for training and test data
path = "YoutubeScrapper/python/out/compresseddata/"

#Assigning batch size
batch_size = 1

#Assigning training data size
train_data_size = 180

#Assigning test data size
test_data_size = 16

#Initializing the generator
training_data = video_generator(path , batch_size=1 , train_data_size = train_data_size , test_data_size = test_data_size , mode = "train")


with tf.device("/CPU:0"):
    model.fit(x=training_data , steps_per_epoch=train_data_size , epochs=1 , callbacks=[tensorboard_callback , checkpoint])

19/180 [==>...........................] - ETA: 1:32:06 - loss: 183.6883 - mse: 183.6883
Epoch 00001: loss improved from inf to 174.71086, saving model to model_1
INFO:tensorflow:Assets written to: model_1\assets
 39/180 [=====>........................] - ETA: 1:34:04 - loss: 229.3042 - mse: 229.3042
Epoch 00001: loss did not improve from 174.71086
 59/180 [========>.....................] - ETA: 1:19:34 - loss: 224.3844 - mse: 224.3844
Epoch 00001: loss did not improve from 174.71086
 79/180 [============>.................] - ETA: 1:06:08 - loss: 204.4345 - mse: 204.4345
Epoch 00001: loss did not improve from 174.71086
 99/180 [===============>..............] - ETA: 55:22 - loss: 206.8901 - mse: 206.8901
Epoch 00001: loss did not improve from 174.71086
119/180 [==================>...........] - ETA: 41:58 - loss: 204.5099 - mse: 204.5099
Epoch 00001: loss did not improve from 174.71086
139/180 [======================>.......] - ETA: 29:24 - loss: 188.7279 - mse: 188.7279
Epoch 00001: lo

In [14]:
model.save_weights("model_1")

In [6]:
#This cell is for testing the model against test data
#Assigning batch size
batch_size = 1

#Assigning training data size
train_data_size = 180

#setting the path for training and test data
path = "YoutubeScrapper/python/out/compresseddata/"

#Assigning test data size
test_data_size = 16

#evaluating data
test_data = video_generator(path , mode = "eval")

with tf.device("/CPU:0"):
    model.evaluate(test_data , steps = test_data_size)

Now testing on example 1
Now testing on example 2
16/16 [==============================] - 42s 3s/step - loss: 0.0859 - mse: 0.0859


In [10]:
results = np.zeros((16 , 2))

for i in range(16):
    with h5py.File(path + "video (" + str(180 + i) + ").hdf5" , "r") as rfile:
        results[i , 0] = np.log(rfile[("label")].value)
        with tf.device("/CPU:0"):
            results[i , 1] = model.predict(rfile["video"].value.reshape((1 , 300 , 360 , 640 , 3)))

print(results)
mse = np.mean(np.square(results[: , 0] - results[: , 1]))
print(mse)

[[2.6431156  3.21012187]
 [3.33283979 3.21012187]
 [3.09572022 3.1907053 ]
 [3.20269007 3.21012187]
 [3.2380752  3.2100563 ]
 [3.29547809 3.21012187]
 [3.17756094 3.21012187]
 [3.24399498 3.21012187]
 [2.58748552 3.21012187]
 [3.14972174 3.21012187]
 [3.69619529 3.21012187]
 [2.91853983 2.99736738]
 [3.43680669 3.21012187]
 [2.93598227 3.21012187]
 [3.48510001 3.21012187]
 [3.08298207 3.21012187]]


In [31]:
values = np.zeros((1 , 180))
for i in range(180):
    with h5py.File(path + "video (" + str(i + 1) + ").hdf5" , "r") as rfile:
        values[0 , i] = np.log(rfile[("label")].value)

print(values)
print(np.mean(values))
data_mean = np.mean(values)
print(np.median(values))

mse = np.zeros((1 , 230))
for i in range(230):
    mse[0 , i] = np.mean(np.square(results[: , 0] - (3.07 + ((i + 1) / 1000))))
    

print(np.argmin(mse))


[[28.35748792 16.7230444  38.95047857 15.08353609 29.05002875 18.12231653
   2.01449275 11.69917479 20.75453048 40.63234323 26.0279537  17.2093242
  20.40818691 44.45222622 49.31662488 41.24226234 29.86590352 49.92768595
  23.98516749 22.99521531 14.12142039 26.95626517 11.49900596 18.4933084
  23.77627628 20.06585062 46.0832495  41.57132353 78.66545455 11.35929648
  17.39303483 18.29154973 21.25208467 37.77776466 52.18252212 31.42992271
  18.33649635 10.57059039 36.41602465 27.77971125 19.53349876 21.5
  24.51872247 20.66812074 54.07916806 64.03370325  4.73398548 23.92034915
  21.25563063 28.03225806 31.35063704 33.17158253 21.82569891  7.82915832
  11.65238034 56.40984268 30.77326671 30.77326671 11.09141147 11.36036036
  25.8745098  12.11494253 20.63751584 37.60014728 28.93274913 26.23485259
  19.024      12.28061423 20.31459436 22.94752775  3.78790229 22.75
  39.96127563 26.37772277 55.49603461  9.14388489 31.1127008  21.01385042
  21.71345595 58.33333333 37.84904259 15.59815774 18.